In [ ]:
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_costs, topside_modules, misc_costs
):
  """Calculates the total decommissioning cost of an offshore platform.

  Args:
    export_pipeline_cost: Decommissioning cost of the export pipeline.
    export_riser_cost: Decommissioning cost of the export riser.
    production_wells: A list of dictionaries, each containing:
      
well_cost: Decommissioning cost of the well
pipeline_cost: Decommissioning cost of the well's pipeline
riser_cost: Decommissioning cost of the well's riser
jacket_costs: A list of decommissioning costs for each jacket.
topside_modules: A list of dictionaries, each containing:
module_cost: Decommissioning cost per unit of the module
unit_count: Number of units for the module
misc_costs: Miscellaneous decommissioning costs.

  Returns:
    The total decommissioning cost.
  """

  total_well_costs = sum(well['well_cost'] + well['pipeline_cost'] + well['riser_cost'] for well in production_wells)
  total_jacket_costs = sum(jacket_costs)
  total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

  total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs

  return total_cost

In [2]:
import pandas as pd

# Load data from CSV files
carbon_df = pd.read_csv('carbon.csv')
hydrogen_df = pd.read_csv('hydrogen.csv')

def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_costs, topside_modules, misc_costs
):
    total_well_costs = sum(well['well_cost'] + well['pipeline_cost'] + well['riser_cost'] for well in production_wells)
    total_jacket_costs = sum(jacket_costs)
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    return total_cost

def calculate_total_ccs_profit(data, year_start, year_end):
    df = data[data['Type'] == 'Onshore']  # Assuming onshore for this example
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        row = df[df['Year'] == year].iloc[0]
        revenue = row['Effective Sale Price /tCO2 ($)']
        cost = row['Cost for capture /tCO2 ($)'] + row['Cost of Transport /tCO2/km ($)'] + row['Cost of Storage /tCO2 ($)']
        profit = revenue - cost
        total_profit += profit
    return total_profit

def calculate_total_hydrogen_profit(data, year_start, year_end):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == year)].iloc[0]
            sell_price = row['Sell Price / kg']
            production_cost = row['Cost of Production /kg ($)']
            transport_cost = row['Cost of Transport /kg/100km']
            total_cost = row['Cost of Installation /kW ($) (Capital Cost)'] + production_cost + transport_cost
            profit = sell_price - total_cost
            total_profit += profit
    return total_profit

def compare_options(decommissioning_cost, ccs_data, hydrogen_data, plant_lifetime, till_year):
    total_ccs_profit = calculate_total_ccs_profit(ccs_data, 2025, till_year)
    total_hydrogen_profit = calculate_total_hydrogen_profit(hydrogen_data, 2025, till_year)

    net_profit_ccs = total_ccs_profit - decommissioning_cost
    net_profit_hydrogen = total_hydrogen_profit - decommissioning_cost

    if decommissioning_cost < max(net_profit_ccs, net_profit_hydrogen):
        return f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f} as it is lower than the potential profits from CCS (${net_profit_ccs:,.2f}) or Hydrogen (${net_profit_hydrogen:,.2f})."

    if net_profit_ccs > net_profit_hydrogen:
        preferred_option = "CCS"
        return f"CCS is preferred due to higher net profit of ${net_profit_ccs:,.2f}. Consider onshore CCS for lower capture costs over time."
    else:
        preferred_option = "Hydrogen"
        return f"Hydrogen is preferred due to higher net profit of ${net_profit_hydrogen:,.2f}. Consider Alkaline Hydrogen for better profitability and efficiency."

# Example usage
decommissioning_cost = 45471500
ccs_data = carbon_df
hydrogen_data = hydrogen_df
plant_lifetime = 2025
till_year = 2055

print(compare_options(decommissioning_cost, ccs_data, hydrogen_data, plant_lifetime, till_year))


ParserError: Error tokenizing data. C error: Expected 7 fields in line 24, saw 8


In [1]:
!pip install pandas



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\Shrikar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd

# Load data from CSV files
carbon_df = pd.read_csv('carbon.csv')
hydrogen_df = pd.read_csv('hydrogen.csv')

def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    print("decom",total_cost)
    return total_cost, pipeline_length

def calculate_total_ccs_profit(data, year_start, year_end, pipeline_length):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        if not row.empty:
            row = row.iloc[0]
            revenue = row['Effective Sale Price /tCO2 ($)']
            capture_cost = row['Cost for capture /tCO2 ($)']
            transport_cost = row['Cost of Transport /tCO2/km ($)'] * pipeline_length
            storage_cost = row['Cost of Storage /tCO2 ($)']
            total_cost = capture_cost + transport_cost + storage_cost
            profit = revenue - total_cost
            total_profit += (profit)*-1
    print("ccs_tot",total_profit)
    return total_profit

def calculate_total_hydrogen_profit(data, year_start, year_end, pipeline_length):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == year)]
            if not row.empty:
                row = row.iloc[0]
                sell_price = row['Sell Price / kg']
                production_cost = row['Cost of Production /kg ($)']
                transport_cost = row['Cost of Transport /kg/100km'] * (pipeline_length / 100)
                capital_cost = row['Cost of Installation /kW ($) (Capital Cost)'] if year == year_start else 0
                total_cost = capital_cost + production_cost + transport_cost
                profit = sell_price - total_cost
                total_profit += (profit)*-1
    print ("h_tot",total_profit)

    return total_profit

def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime, till_year
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate profits
    total_ccs_profit = calculate_total_ccs_profit(ccs_data, plant_lifetime, till_year, pipeline_length)
    total_hydrogen_profit = calculate_total_hydrogen_profit(hydrogen_data, plant_lifetime, till_year, pipeline_length)

    # Net profits
    net_profit_ccs = total_ccs_profit - decommissioning_cost
    net_profit_hydrogen = total_hydrogen_profit - decommissioning_cost

    # Decision
    if decommissioning_cost < max(net_profit_ccs, net_profit_hydrogen):
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f} as it is lower than the potential profits "
                f"from CCS (${net_profit_ccs:,.2f}) or Hydrogen (${net_profit_hydrogen:,.2f}).")

    if net_profit_ccs > net_profit_hydrogen:
        preferred_option = "CCS"
        return (f"CCS is preferred due to higher net profit of ${net_profit_ccs:,.2f}. "
                f"Consider onshore CCS for lower capture costs over time. The profit analysis shows that the CCS option yields a greater "
                f"net profit compared to Hydrogen, especially when using onshore facilities where capture and storage costs are lower.")
    else:
        preferred_option = "Hydrogen"
        return (f"Hydrogen is preferred due to higher net profit of ${net_profit_hydrogen:,.2f}. "
                f"Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a "
                f"more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.")

# Example usage
# User needs to provide these values
export_pipeline_cost = 10000000  # Example value
export_riser_cost = 5000000  # Example value
production_wells = [{
    'well_cost': 2000000,
    'well_cost_unit': 1,
    'pipeline_cost': 1000000,
    'pipeline_length': 50,
    'riser_cost': 500000,
    'riser_length': 50,
    'requires_pipeline': True
}]
jacket_cost = 1500000  # Example value
jacket_units = 5  # Example value
topside_modules = [{
    'module_cost': 2000000,
    'unit_count': 2
}]
misc_costs = 500000  # Example value

plant_lifetime = 2025
till_year = 2055

print(compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, plant_lifetime, till_year
))


ParserError: Error tokenizing data. C error: Expected 7 fields in line 24, saw 8


In [10]:
import pandas as pd

# Load data from CSV files
carbon_df = pd.read_csv('carbon.csv')
hydrogen_df = pd.read_csv('hydrogen.csv')

# Facility data
facility_life_remaining = 2040 - 2024  # assuming current year is 2024
co2_production = 1.2  # MTPA
facility_temperature = (10, 110)  # 10°C - 110°C
facility_pressure = 100  # 100 barg

# Feasibility analysis placeholders
ccs_feasibility = True  # Placeholder for actual analysis
hydrogen_feasibility = True  # Placeholder for actual analysis

# Long-term suitability analysis
if facility_temperature[0] <= 25 and facility_temperature[1] >= 100 and facility_pressure >= 100:
    long_term_suitability = "CCS"
else:
    long_term_suitability = "Hydrogen Production"

# Calculate decommissioning cost
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    return total_cost, pipeline_length

# Calculate total CCS profit
def calculate_total_ccs_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        if not row.empty:
            row = row.iloc[0]
            revenue = row['Effective Sale Price /tCO2 ($)']
            capture_cost = row['Cost for capture /tCO2 ($)']
            transport_cost = row['Cost of Transport /tCO2/km ($)'] * pipeline_length
            storage_cost = row['Cost of Storage /tCO2 ($)']
            total_cost = capture_cost + transport_cost + storage_cost
            profit = revenue - total_cost
            total_profit += profit * co2_production
    return total_profit

# Calculate total Hydrogen profit
def calculate_total_hydrogen_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == year)]
            if not row.empty:
                row = row.iloc[0]
                sell_price = row['Sell Price / kg']
                production_cost = row['Cost of Production /kg ($) (Operational)']
                transport_cost = row['Cost of Transport /kg/100km'] * (pipeline_length / 100)
                capital_cost = row['Cost for Installation /kW ($) (Capital Cost)'] if year == year_start else 0
                total_cost = capital_cost + production_cost + transport_cost
                profit = sell_price - total_cost
                total_profit += profit * co2_production / row['Efficiency (kWh/kgH2)']
    return total_profit

# Compare options
def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime, till_year
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate profits
    total_ccs_profit = calculate_total_ccs_profit(ccs_data, plant_lifetime, till_year, pipeline_length, co2_production)
    total_hydrogen_profit = calculate_total_hydrogen_profit(hydrogen_data, plant_lifetime, till_year, pipeline_length, co2_production)

    # Net profits
    net_profit_ccs = total_ccs_profit - decommissioning_cost
    net_profit_hydrogen = total_hydrogen_profit - decommissioning_cost

    # Ensure profits are non-negative
    net_profit_ccs = max(net_profit_ccs, 0)
    net_profit_hydrogen = max(net_profit_hydrogen, 0)

    # Decision
    if decommissioning_cost < max(net_profit_ccs, net_profit_hydrogen):
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f} as it is lower than the potential profits "
                f"from CCS (${net_profit_ccs:,.2f}) or Hydrogen (${net_profit_hydrogen:,.2f}).")

    if net_profit_ccs > net_profit_hydrogen:
        preferred_option = "CCS"
        return (f"CCS is preferred due to higher net profit of ${net_profit_ccs:,.2f}. "
                f"Consider onshore CCS for lower capture costs over time. The profit analysis shows that the CCS option yields a greater "
                f"net profit compared to Hydrogen, especially when using onshore facilities where capture and storage costs are lower.")
    else:
        preferred_option = "Hydrogen"
        return (f"Hydrogen is preferred due to higher net profit of ${net_profit_hydrogen:,.2f}. "
                f"Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a "
                f"more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.")

# Example usage
export_pipeline_cost = 10000000  # Example value
export_riser_cost = 5000000  # Example value
production_wells = [{
    'well_cost': 2000000,
    'well_cost_unit': 1,
    'pipeline_cost': 1000000,
    'pipeline_length': 50,
    'riser_cost': 500000,
    'riser_length': 50,
    'requires_pipeline': True
}]
jacket_cost = 1500000  # Example value
jacket_units = 5  # Example value
topside_modules = [{
    'module_cost': 2000000,
    'unit_count': 2
}]
misc_costs = 500000  # Example value

plant_lifetime = 2025
till_year = 2055

print(compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, plant_lifetime, till_year
))


Hydrogen is preferred due to higher net profit of $0.00. Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.


In [8]:
print(hydrogen_df.columns)


Index(['Type', 'Year', 'Cost for Installation /kW ($) (Capital Cost)',
       'Cost of Production /kg ($) (Operational)',
       'Cost of Transport /kg/100km', 'Sell Price / kg',
       'Efficiency (kWh/kgH2)'],
      dtype='object')


In [11]:
import pandas as pd

# Load data from CSV files
carbon_df = pd.read_csv('carbon.csv')
hydrogen_df = pd.read_csv('hydrogen.csv')

# Facility data
facility_life_remaining = 2040 - 2024  # assuming current year is 2024
co2_production = 1.2  # MTPA
facility_temperature = (10, 110)  # 10°C - 110°C
facility_pressure = 100  # 100 barg

# Feasibility analysis placeholders
ccs_feasibility = True  # Placeholder for actual analysis
hydrogen_feasibility = True  # Placeholder for actual analysis

# Long-term suitability analysis
if facility_temperature[0] <= 25 and facility_temperature[1] >= 100 and facility_pressure >= 100:
    long_term_suitability = "CCS"
else:
    long_term_suitability = "Hydrogen Production"

# Calculate decommissioning cost
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    return total_cost, pipeline_length

# Calculate total CCS profit
def calculate_total_ccs_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        if not row.empty:
            row = row.iloc[0]
            revenue = row.get('Effective Sale Price /tCO2 ($)', 0)
            capture_cost = row.get('Cost for capture /tCO2 ($)', 0)
            transport_cost = row.get('Cost of Transport /tCO2/km ($)', 0) * pipeline_length
            storage_cost = row.get('Cost of Storage /tCO2 ($)', 0)
            total_cost = capture_cost + transport_cost + storage_cost
            profit = revenue - total_cost
            total_profit += profit * co2_production
    return total_profit

# Calculate total Hydrogen profit
def calculate_total_hydrogen_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == year)]
            if not row.empty:
                row = row.iloc[0]
                sell_price = row.get('Sell Price / kg', 0)
                production_cost = row.get('Cost of Production /kg ($) (Operational)', 0)
                transport_cost = row.get('Cost of Transport /kg/100km', 0) * (pipeline_length / 100)
                capital_cost = row.get('Cost for Installation /kW ($) (Capital Cost)', 0) if year == year_start else 0
                total_cost = capital_cost + production_cost + transport_cost
                profit = sell_price - total_cost
                total_profit += profit * co2_production / row.get('Efficiency (kWh/kgH2)', 1)
    return total_profit

# Compare options
def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime, till_year
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate profits
    total_ccs_profit = calculate_total_ccs_profit(ccs_data, plant_lifetime, till_year, pipeline_length, co2_production)
    total_hydrogen_profit = calculate_total_hydrogen_profit(hydrogen_data, plant_lifetime, till_year, pipeline_length, co2_production)

    # Net profits
    net_profit_ccs = max(total_ccs_profit - decommissioning_cost, 0)
    net_profit_hydrogen = max(total_hydrogen_profit - decommissioning_cost, 0)

    # Decision
    if decommissioning_cost < max(net_profit_ccs, net_profit_hydrogen):
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f} as it is lower than the potential profits "
                f"from CCS (${net_profit_ccs:,.2f}) or Hydrogen (${net_profit_hydrogen:,.2f}).")

    if net_profit_ccs > net_profit_hydrogen:
        preferred_option = "CCS"
        return (f"CCS is preferred due to higher net profit of ${net_profit_ccs:,.2f}. "
                f"Consider onshore CCS for lower capture costs over time. The profit analysis shows that the CCS option yields a greater "
                f"net profit compared to Hydrogen, especially when using onshore facilities where capture and storage costs are lower.")
    else:
        preferred_option = "Hydrogen"
        return (f"Hydrogen is preferred due to higher net profit of ${net_profit_hydrogen:,.2f}. "
                f"Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a "
                f"more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.")

# Example usage
export_pipeline_cost = 10000000  # Example value
export_riser_cost = 5000000  # Example value
production_wells = [{
    'well_cost': 2000000,
    'well_cost_unit': 1,
    'pipeline_cost': 1000000,
    'pipeline_length': 50,
    'riser_cost': 500000,
    'riser_length': 50,
    'requires_pipeline': True
}]
jacket_cost = 1500000  # Example value
jacket_units = 5  # Example value
topside_modules = [{
    'module_cost': 2000000,
    'unit_count': 2
}]
misc_costs = 500000  # Example value

plant_lifetime = 2025
till_year = 2055

print(compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, plant_lifetime, till_year
))


C:\Users\Shrikar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Shrikar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Hydrogen is preferred due to higher net profit of $0.00. The ROI for Hydrogen is -100.00%. Alkaline Hydrogen technology is recommended for better profitability and efficiency. It offers a CO2 reduction impact of 0.00 tons, with a more favorable balance of production and transport costs compared to PEM technology. Forecasted Hydrogen prices indicate potential for continued profitability.


In [12]:
import pandas as pd

# Load data from CSV files
carbon_df = pd.read_csv('carbon.csv')
hydrogen_df = pd.read_csv('hydrogen.csv')

# Facility data
facility_life_remaining = 2040 - 2024  # assuming current year is 2024
co2_production = 1.2  # MTPA
facility_temperature = (10, 110)  # 10°C - 110°C
facility_pressure = 100  # 100 barg

# Feasibility analysis placeholders
ccs_feasibility = True  # Placeholder for actual analysis
hydrogen_feasibility = True  # Placeholder for actual analysis

# Long-term suitability analysis
if facility_temperature[0] <= 25 and facility_temperature[1] >= 100 and facility_pressure >= 100:
    long_term_suitability = "CCS"
else:
    long_term_suitability = "Hydrogen Production"

# Calculate decommissioning cost
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    return total_cost, pipeline_length

# Calculate total CCS profit
def calculate_total_ccs_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        if not row.empty:
            row = row.iloc[0]
            revenue = row.get('Effective Sale Price /tCO2 ($)', 0)
            capture_cost = row.get('Cost for capture /tCO2 ($)', 0)
            transport_cost = row.get('Cost of Transport /tCO2/km ($)', 0) * pipeline_length
            storage_cost = row.get('Cost of Storage /tCO2 ($)', 0)
            total_cost = capture_cost + transport_cost + storage_cost
            profit = revenue - total_cost
            total_profit += profit * co2_production
    return total_profit

# Calculate total Hydrogen profit
def calculate_total_hydrogen_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == year)]
            if not row.empty:
                row = row.iloc[0]
                sell_price = row.get('Sell Price / kg', 0)
                production_cost = row.get('Cost of Production /kg ($) (Operational)', 0)
                transport_cost = row.get('Cost of Transport /kg/100km', 0) * (pipeline_length / 100)
                capital_cost = row.get('Cost for Installation /kW ($) (Capital Cost)', 0) if year == year_start else 0
                total_cost = capital_cost + production_cost + transport_cost
                profit = sell_price - total_cost
                total_profit += profit * co2_production / row.get('Efficiency (kWh/kgH2)', 1)
    return total_profit

# Compare options
def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime, till_year
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate profits
    total_ccs_profit = calculate_total_ccs_profit(ccs_data, plant_lifetime, till_year, pipeline_length, co2_production)
    total_hydrogen_profit = calculate_total_hydrogen_profit(hydrogen_data, plant_lifetime, till_year, pipeline_length, co2_production)

    # Net profits
    net_profit_ccs = max(total_ccs_profit - decommissioning_cost, 0)
    net_profit_hydrogen = max(total_hydrogen_profit - decommissioning_cost, 0)

    # Decision
    if decommissioning_cost < max(net_profit_ccs, net_profit_hydrogen):
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f} as it is lower than the potential profits "
                f"from CCS (${net_profit_ccs:,.2f}) or Hydrogen (${net_profit_hydrogen:,.2f}).")

    if net_profit_ccs > net_profit_hydrogen:
        preferred_option = "CCS"
        return (f"CCS is preferred due to higher net profit of ${net_profit_ccs:,.2f}. "
                f"Consider onshore CCS for lower capture costs over time. The profit analysis shows that the CCS option yields a greater "
                f"net profit compared to Hydrogen, especially when using onshore facilities where capture and storage costs are lower.")
    else:
        preferred_option = "Hydrogen"
        return (f"Hydrogen is preferred due to higher net profit of ${net_profit_hydrogen:,.2f}. "
                f"Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a "
                f"more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.")

# Example usage
export_pipeline_cost = 10000000  # Example value
export_riser_cost = 5000000  # Example value
production_wells = [{
    'well_cost': 2000000,
    'well_cost_unit': 1,
    'pipeline_cost': 1000000,
    'pipeline_length': 50,
    'riser_cost': 500000,
    'riser_length': 50,
    'requires_pipeline': True
}]
jacket_cost = 1500000  # Example value
jacket_units = 5  # Example value
topside_modules = [{
    'module_cost': 2000000,
    'unit_count': 2
}]
misc_costs = 500000  # Example value

plant_lifetime = 2025
till_year = 2055

print(compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, plant_lifetime, till_year
))


Hydrogen is preferred due to higher net profit of $0.00. Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.


In [15]:
import pandas as pd

# Load data from CSV files
carbon_df = pd.read_csv('carbon.csv')
hydrogen_df = pd.read_csv('hydrogen.csv')

# Facility data
current_year = 2024
facility_life_remaining = 2040 - current_year  # 2040 is the end of oil and gas extraction
facility_end_of_life = 2045  # end of life for decommissioning or repurposing
co2_production = 5  # MTPA
facility_temperature = (10, 110)  # 10°C - 110°C
facility_pressure = 100  # 100 barg

# Calculate decommissioning cost
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, reinjection_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_cost = 0
    
    total_cost += export_pipeline_cost * 400  # 400 km
    total_cost += export_riser_cost * 80  # 80 m
    total_cost += sum(well['cost'] for well in production_wells)
    total_cost += sum(well['cost'] for well in reinjection_wells)
    total_cost += jacket_cost * jacket_units
    total_cost += sum(module['cost'] for module in topside_modules)
    total_cost += misc_costs
    
    return total_cost

# Calculate total CCS profit
def calculate_total_ccs_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        if not row.empty:
            row = row.iloc[0]
            revenue = row.get('Effective Sale Price /tCO2 ($)', 0)
            capture_cost = row.get('Cost for capture /tCO2 ($)', 0)
            transport_cost = row.get('Cost of Transport /tCO2/km ($)', 0) * pipeline_length
            storage_cost = row.get('Cost of Storage /tCO2 ($)', 0)
            total_cost = capture_cost + transport_cost + storage_cost
            profit = revenue - total_cost
            total_profit += profit * co2_production * 1e6  # convert MTPA to tPA
    return total_profit

# Calculate total Hydrogen profit
def calculate_total_hydrogen_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == year)]
            if not row.empty:
                row = row.iloc[0]
                sell_price = row.get('Sell Price / kg', 0)
                production_cost = row.get('Cost of Production /kg ($) (Operational)', 0)
                transport_cost = row.get('Cost of Transport /kg/100km', 0) * (pipeline_length / 100)
                capital_cost = row.get('Cost for Installation /kW ($) (Capital Cost)', 0) if year == year_start else 0
                total_cost = capital_cost + production_cost + transport_cost
                profit = sell_price - total_cost
                total_profit += profit * co2_production * 1e6 / row.get('Efficiency (kWh/kgH2)', 1)  # convert MTPA to kgPA
    return total_profit

# Decommissioning cost components
export_pipeline_cost = 472500  # per km
export_riser_cost = 11000  # per m
production_wells = [{'cost': 1100000}] * 5  # 5 wells
reinjection_wells = [{'cost': 1050000}] * 4  # 4 wells
jacket_cost = 172500000  # per jacket
jacket_units = 2
topside_modules = [
    {'cost': 210000000},  # Topside Module 1
    {'cost': 525000000},  # Topside Module 2
    {'cost': 88000000},   # Topside Module 3
    {'cost': 10500000},   # Topside Module 4
    {'cost': 126000000},  # Topside Module 5
    {'cost': 330000000},  # Topside Module 6
]
misc_costs = 5500000  # miscellaneous costs

# Example usage
plant_lifetime = current_year + 3  # current year + remaining extraction years
till_year = 2045  # end of life for decommissioning or repurposing

# Calculate decommissioning cost and pipeline length
decommissioning_cost = calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, reinjection_wells, jacket_cost, jacket_units, topside_modules, misc_costs
)

# Calculate profits
total_ccs_profit = calculate_total_ccs_profit(carbon_df, plant_lifetime, till_year, 400, co2_production)
total_hydrogen_profit = calculate_total_hydrogen_profit(hydrogen_df, plant_lifetime, till_year, 400, co2_production)

# Net profits
net_profit_ccs = total_ccs_profit - decommissioning_cost
net_profit_hydrogen = total_hydrogen_profit - decommissioning_cost

# Decision
if net_profit_ccs > net_profit_hydrogen:
    preferred_option = "CCS"
    result = f"CCS is preferred due to higher net profit of ${net_profit_ccs:,.2f}."
else:
    preferred_option = "Hydrogen"
    result = f"Hydrogen is preferred due to higher net profit of ${net_profit_hydrogen:,.2f}."

print(result)


Hydrogen is preferred due to higher net profit of $-1,839,580,000.00.


In [25]:
import pandas as pd

# Load the data
carbon_data_path = 'carbon.csv'
hydrogen_data_path = 'hydrogen.csv'
carbon_df = pd.read_csv(carbon_data_path)
hydrogen_df = pd.read_csv(hydrogen_data_path)

# Define constants
CO2_PRODUCTION = 5e6  # 1.2 million tonnes per annum
EXTRACTION_PERIOD_YEARS = 3
PLANT_LIFETIME_START = 2025
PLANT_LIFETIME_END = 2040

# Decommissioning costs
export_pipeline_cost = 825000 * 400  # Cost for 400 km pipeline
export_riser_cost = 16500 * 80  # Cost for 80 m riser
production_wells = [{
    'well_cost': 4600000,
    'well_cost_unit': 5,
    'pipeline_cost': 1000000,
    'pipeline_length': 50,
    'riser_cost': 500000,
    'riser_length': 50,
    'requires_pipeline': True
}]
jacket_cost = 575000000  # Cost for 2 jackets
jacket_units = 2  # Number of jackets
topside_modules = [
    {'module_cost': 440000000, 'unit_count': 1},
    {'module_cost': 920000000, 'unit_count': 1},
    {'module_cost': 172500000, 'unit_count': 1},
    {'module_cost': 165000000, 'unit_count': 1},
    {'module_cost': 330000000, 'unit_count': 1},
    {'module_cost': 460000000, 'unit_count': 1}
]
misc_costs = 80500000  # Miscellaneous costs

# Define functions
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    print("decom",total_cost)
    return total_cost, pipeline_length

def calculate_total_ccs_profit(data, year_start, year_end, pipeline_length, co2_production):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        if not row.empty:
            row = row.iloc[0]
            revenue = row.get('Effective Sale Price /tCO2 ($)', 0)
            capture_cost = row.get('Cost for capture /tCO2 ($)', 0)
            transport_cost = row.get('Cost of Transport /tCO2/km ($)', 0) * pipeline_length
            storage_cost = row.get('Cost of Storage /tCO2 ($)', 0)
            total_cost = capture_cost + transport_cost + storage_cost
            profit = revenue - total_cost
            total_profit += profit * co2_production
    print("co2",profit)
    return total_profit

def calculate_total_hydrogen_profit(data, year_start, year_end, pipeline_length):
    total_profit = 0
    for year in range(year_start, year_end + 1, 5):
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == year)]
            if not row.empty:
                row = row.iloc[0]
                sell_price = row.get('Sell Price / kg', 0)
                production_cost = row.get('Cost of Production /kg ($) (Operational)', 0)
                transport_cost_per_100km = row.get('Cost of Transport /kg/100km', 0)
                capital_cost_per_kw = row.get('Cost for Installation /kW ($) (Capital Cost)', 0)
                efficiency = row.get('Efficiency (kWh/kgH2)', 1)


                transport_cost = transport_cost_per_100km * (pipeline_length / 100)


                capital_cost = capital_cost_per_kw if year == year_start else 0

                # Calculate total cost per kg of hydrogen
                total_cost = production_cost + transport_cost + capital_cost / (efficiency * 1000)  # Adjusting for capital cost over production efficiency

                # Calculate profit
                profit_per_kg = sell_price - total_cost
                total_profit += profit_per_kg * co2_production / efficiency  # Adjusting for efficiency in production
    return total_profit


def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime_start, plant_lifetime_end, co2_production
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )

    total_ccs_profit = calculate_total_ccs_profit(ccs_data, plant_lifetime_start, plant_lifetime_end, pipeline_length, co2_production)
    total_hydrogen_profit = calculate_total_hydrogen_profit(hydrogen_data, plant_lifetime_start, plant_lifetime_end, pipeline_length)

    # Net profits
    net_profit_ccs = max(total_ccs_profit - decommissioning_cost, 0)
    net_profit_hydrogen = max(total_hydrogen_profit - decommissioning_cost, 0)

    # Decision
    if decommissioning_cost < max(net_profit_ccs, net_profit_hydrogen):
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f} as it is lower than the potential profits "
                f"from CCS (${net_profit_ccs:,.2f}) or Hydrogen (${net_profit_hydrogen:,.2f}).")

    if net_profit_ccs > net_profit_hydrogen:
        return (f"CCS is preferred due to higher net profit of ${net_profit_ccs:,.2f}. "
                f"Consider onshore CCS for lower capture costs over time. The profit analysis shows that the CCS option yields a greater "
                f"net profit compared to Hydrogen, especially when using onshore facilities where capture and storage costs are lower.")
    else:
        return (f"Hydrogen is preferred due to higher net profit of ${net_profit_hydrogen:,.2f}. "
                f"Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a "
                f"more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.")

result = compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, PLANT_LIFETIME_START, PLANT_LIFETIME_END, CO2_PRODUCTION
)

print("sachin",result)
def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime_start, plant_lifetime_end, co2_production, extraction_period_years
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate profits
    total_ccs_profit = calculate_total_ccs_profit(ccs_data, plant_lifetime_start, plant_lifetime_end, pipeline_length, co2_production)
    total_hydrogen_profit = calculate_total_hydrogen_profit(hydrogen_data, plant_lifetime_start, plant_lifetime_end, pipeline_length, co2_production, extraction_period_years)

    # Net profits
    net_profit_ccs = max(total_ccs_profit - decommissioning_cost, 0)
    net_profit_hydrogen = max(total_hydrogen_profit - decommissioning_cost, 0)

    # Decision
    if decommissioning_cost < max(net_profit_ccs, net_profit_hydrogen):
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f} as it is lower than the potential profits "
                f"from CCS (${net_profit_ccs:,.2f}) or Hydrogen (${net_profit_hydrogen:,.2f}).")

    if net_profit_ccs > net_profit_hydrogen:
        return (f"CCS is preferred due to higher net profit of ${net_profit_ccs:,.2f}. "
                f"Consider onshore CCS for lower capture costs over time. The profit analysis shows that the CCS option yields a greater "
                f"net profit compared to Hydrogen, especially when using onshore facilities where capture and storage costs are lower.")
    else:
        return (f"Hydrogen is preferred due to higher net profit of ${net_profit_hydrogen:,.2f}. "
                f"Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a "
                f"more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.")

# Compare options and provide insights
result = compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, PLANT_LIFETIME_START, PLANT_LIFETIME_END, CO2_PRODUCTION, EXTRACTION_PERIOD_YEARS
)

print(result)


decom 4147320000
co2 -68.4
sachin Hydrogen is preferred due to higher net profit of $0.00. Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a more favorable balance of production and transport costs, yielding higher profits compared to PEM technology.
decom 4147320000
co2 -68.4


TypeError: calculate_total_hydrogen_profit() takes 4 positional arguments but 6 were given

In [41]:
carbon_df['Year'] = carbon_df['Year'].astype(int)


In [45]:
import pandas as pd

# Load the data
carbon_data_path = 'carbon.csv'
hydrogen_data_path = 'hydrogen.csv'
carbon_df = pd.read_csv(carbon_data_path)
hydrogen_df = pd.read_csv(hydrogen_data_path)

# Define constants
CO2_PRODUCTION = 1.2e6  # 1.2 million tonnes per annum
PLANT_LIFETIME_START = 2025
PLANT_LIFETIME_END = 2055

# Decommissioning costs
export_pipeline_cost = 275000 * 20  # Cost for 400 km pipeline
export_riser_cost = 11500 * 15  # Cost for 80 m riser
production_wells = [{
    'well_cost': 6900000,
    'well_cost_unit': 1,
    'pipeline_cost': 210000,
    'pipeline_length': 2,
    'riser_cost': 10500,
    'riser_length': 15,
    'requires_pipeline': True,
    'max_sequestration_rate': 2e6  # Maximum sequestration rate per well in tonnes per annum
},
{
    'well_cost': 4400000,
    'well_cost_unit': 1,
    'pipeline_cost': 230000,
    'pipeline_length': 1,
    'riser_cost': 11500,
    'riser_length': 15,
    'requires_pipeline': True,
    'max_sequestration_rate': 2e6  # Maximum sequestration rate per well in tonnes per annum
}]
jacket_cost = 15750000  # Cost for 2 jackets
jacket_units = 1  # Number of jackets
topside_modules = [
    {'module_cost': 1575000, 'unit_count': 1},
    {'module_cost': 2750000, 'unit_count': 1},
    {'module_cost': 2100000, 'unit_count': 1},
    {'module_cost': 3069000, 'unit_count': 1},
    {'module_cost': 575000, 'unit_count': 1},
]
misc_costs = 0  # Miscellaneous costs

# Define functions
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    print("decom", total_cost)
    return total_cost, pipeline_length

def calculate_net_ccs_cost(data, year_start, year_end, pipeline_length, production_wells):
    total_net_cost = 0
    total_sequestration_capacity = sum(well['max_sequestration_rate'] for well in production_wells)
    
    for year in range(year_start, year_end + 1, 5):
        print(f"Year: {year}")
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        print(f"Filtered Data for Year {year}:")
        print(row)
        if not row.empty:
            row = row.iloc[0]
            revenue = row.get('Sale_price', 1)
            capture_cost = row.get('Cost_capture', 1)
            transport_cost = row.get('Cost_transport', 1) * pipeline_length
            storage_cost = row.get('Cost_storage', 1)
            print("rev", revenue, capture_cost, transport_cost, storage_cost)
            total_cost = capture_cost + transport_cost + storage_cost
            print(total_cost)
            net_cost = total_cost - revenue
            total_net_cost += net_cost * total_sequestration_capacity
            print(net_cost, total_net_cost)
    
    print("ccs", total_net_cost)
    return total_net_cost

def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime_start, plant_lifetime_end, co2_production
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate net costs
    total_net_ccs_cost = calculate_net_ccs_cost(ccs_data, plant_lifetime_start, plant_lifetime_end, pipeline_length, production_wells)

    # Decision
    if decommissioning_cost < total_net_ccs_cost:
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f}, "
                f"as it is lower than the net costs of CCS (${total_net_ccs_cost:,.2f}).")

# Compare options and provide insights
result = compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, PLANT_LIFETIME_START, PLANT_LIFETIME_END, CO2_PRODUCTION
)

print(result)


decom 43771500
Year: 2025
Filtered Data for Year 2025:
      Type  Year  Cost for capture /tCO2 ($)  Cost of Transport /tCO2/km ($)  \
0  Onshore  2025                       113.0                             2.2   

   Cost of Storage /tCO2 ($)  Effective Sale Price /tCO2 ($)  
0                       34.1                            39.0  
rev 1 1 3 1
5
4 16000000.0
Year: 2030
Filtered Data for Year 2030:
      Type  Year  Cost for capture /tCO2 ($)  Cost of Transport /tCO2/km ($)  \
1  Onshore  2030                       105.0                             2.0   

   Cost of Storage /tCO2 ($)  Effective Sale Price /tCO2 ($)  
1                       29.5                            55.0  
rev 1 1 3 1
5
4 32000000.0
Year: 2035
Filtered Data for Year 2035:
      Type  Year  Cost for capture /tCO2 ($)  Cost of Transport /tCO2/km ($)  \
2  Onshore  2035                        92.0                             1.8   

   Cost of Storage /tCO2 ($)  Effective Sale Price /tCO2 ($)  
2            

In [39]:
import pandas as pd

# Load the data
carbon_data_path = 'carbon.csv'
hydrogen_data_path = 'hydrogen.csv'
carbon_df = pd.read_csv(carbon_data_path)
hydrogen_df = pd.read_csv(hydrogen_data_path)

# Define constants
CO2_PRODUCTION = 1.2e6  # 1.2 million tonnes per annum
PLANT_LIFETIME_START = 2035
PLANT_LIFETIME_END = 2055

# Decommissioning costs
export_pipeline_cost = 275000 * 20 # Cost for 400 km pipeline
export_riser_cost = 11500 * 15  # Cost for 80 m riser
production_wells = [{
    'well_cost': 6900000,
    'well_cost_unit': 1,
    'pipeline_cost': 210000,
    'pipeline_length': 2,
    'riser_cost': 10500,
    'riser_length': 15,
    'requires_pipeline': True,
    'max_sequestration_rate': 2e6  # Maximum sequestration rate per well in tonnes per annum
},
{
    'well_cost': 4400000,
    'well_cost_unit': 1,
    'pipeline_cost': 230000,
    'pipeline_length': 1,
    'riser_cost': 11500,
    'riser_length': 15,
    'requires_pipeline': True,
    'max_sequestration_rate': 2e6  # Maximum sequestration rate per well in tonnes per annum
}]
jacket_cost = 15750000 # Cost for 2 jackets
jacket_units = 1  # Number of jackets
topside_modules = [
    {'module_cost': 1575000, 'unit_count': 1},
    {'module_cost': 2750000, 'unit_count': 1},
    {'module_cost': 2100000, 'unit_count': 1},
    {'module_cost': 3069000, 'unit_count': 1},
    {'module_cost': 575000, 'unit_count': 1},
   
]
misc_costs = 0  # Misc0ellaneous costs

 # Miscellaneous costs

# Define functions
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    print("decom",total_cost)
    return total_cost, pipeline_length

def calculate_net_ccs_cost(data, year_start, year_end, pipeline_length, production_wells):
    total_net_cost = 0
    total_sequestration_capacity = sum(well['max_sequestration_rate'] for well in production_wells)
    
    for year in range(year_start, year_end + 1, 5):
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        if not row.empty:
            row = row.iloc[0]
            revenue = row.get('Effective Sale Price /tCO2 ($)', 0)
            capture_cost = row.get('Cost for capture /tCO2 ($)', 0)
            transport_cost = row.get('Cost of Transport /tCO2/km ($)', 0) * pipeline_length
            storage_cost = row.get('Cost of Storage /tCO2 ($)', 0)
            total_cost = capture_cost + transport_cost + storage_cost
            net_cost = total_cost - revenue
            total_net_cost += net_cost * total_sequestration_capacity
    print("ccs",total_net_cost)
    return total_net_cost

def calculate_net_hydrogen_cost(data, year_start, year_end, pipeline_length):
    total_net_cost = 0
    for year in range(year_start, year_end + 1, 5):
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == year)]
            if not row.empty:
                row = row.iloc[0]
                sell_price = row.get('Sell Price / kg', 0)
                production_cost = row.get('Cost of Production /kg ($) (Operational)', 0)
                transport_cost_per_100km = row.get('Cost of Transport /kg/100km', 0)
                capital_cost_per_kw = row.get('Cost for Installation /kW ($) (Capital Cost)', 0)
                efficiency = row.get('Efficiency (kWh/kgH2)', 1)
                
                # Calculate transport cost based on pipeline length
                transport_cost = transport_cost_per_100km * (pipeline_length / 100)
                
                # Capital cost should be considered for the initial year only
                capital_cost = capital_cost_per_kw if year == year_start else 0

                # Calculate total cost per kg of hydrogen
                total_cost = production_cost + transport_cost + capital_cost / (efficiency * 1000)  # Adjusting for capital cost over production efficiency

                # Calculate net cost
                net_cost_per_kg = total_cost - sell_price
                total_net_cost += net_cost_per_kg * CO2_PRODUCTION / efficiency
    print("h2",total_net_cost)          # Adjusting for efficiency in production
    return total_net_cost

def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime_start, plant_lifetime_end, co2_production
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate net costs
    total_net_ccs_cost = calculate_net_ccs_cost(ccs_data, plant_lifetime_start, plant_lifetime_end, pipeline_length, production_wells)
    total_net_hydrogen_cost = calculate_net_hydrogen_cost(hydrogen_data, plant_lifetime_start, plant_lifetime_end, pipeline_length)

    # Decision
    if decommissioning_cost < total_net_ccs_cost and decommissioning_cost < total_net_hydrogen_cost:
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f}, "
                f"as it is lower than the net costs of CCS (${total_net_ccs_cost:,.2f}) and Hydrogen (${total_net_hydrogen_cost:,.2f}).")

    if total_net_ccs_cost < total_net_hydrogen_cost:
        return (f"CCS is preferred with a net cost of ${total_net_ccs_cost:,.2f}. "
                f"Consider onshore CCS for lower capture costs over time. The net cost analysis shows that CCS is more cost-effective "
                f"compared to Hydrogen production.")
    else:
        return (f"Hydrogen is preferred with a net cost of ${total_net_hydrogen_cost:,.2f}. "
                f"Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers "
                f"a more favorable balance of production and transport costs, yielding lower net costs compared to PEM technology.")

# Compare options and provide insights
result = compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, PLANT_LIFETIME_START, PLANT_LIFETIME_END, CO2_PRODUCTION
)

print(result)


decom 43771500
ccs -925199999.9999998
h2 -317434.22674827947
CCS is preferred with a net cost of $-925,200,000.00. Consider onshore CCS for lower capture costs over time. The net cost analysis shows that CCS is more cost-effective compared to Hydrogen production.


In [46]:
import pandas as pd

# Load the data
carbon_data_path = 'carbon.csv'
hydrogen_data_path = 'hydrogen.csv'
carbon_df = pd.read_csv(carbon_data_path)
hydrogen_df = pd.read_csv(hydrogen_data_path)

# Define constants
CO2_PRODUCTION = 1.2e6  # 1.2 million tonnes per annum
PLANT_LIFETIME_START = 2025
PLANT_LIFETIME_END = 2055

# Decommissioning costs
export_pipeline_cost = 275000 * 20  # Cost for 400 km pipeline
export_riser_cost = 11500 * 15  # Cost for 80 m riser
production_wells = [{
    'well_cost': 6900000,
    'well_cost_unit': 1,
    'pipeline_cost': 210000,
    'pipeline_length': 2,
    'riser_cost': 10500,
    'riser_length': 15,
    'requires_pipeline': True,
    'max_sequestration_rate': 2e6  # Maximum sequestration rate per well in tonnes per annum
},
{
    'well_cost': 4400000,
    'well_cost_unit': 1,
    'pipeline_cost': 230000,
    'pipeline_length': 1,
    'riser_cost': 11500,
    'riser_length': 15,
    'requires_pipeline': True,
    'max_sequestration_rate': 2e6  # Maximum sequestration rate per well in tonnes per annum
}]
jacket_cost = 15750000  # Cost for 2 jackets
jacket_units = 1  # Number of jackets
topside_modules = [
    {'module_cost': 1575000, 'unit_count': 1},
    {'module_cost': 2750000, 'unit_count': 1},
    {'module_cost': 2100000, 'unit_count': 1},
    {'module_cost': 3069000, 'unit_count': 1},
    {'module_cost': 575000, 'unit_count': 1},
]
misc_costs = 0  # Miscellaneous costs

# Define functions
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    print("decom", total_cost)
    return total_cost, pipeline_length

def calculate_net_ccs_cost(data, year_start, year_end, pipeline_length, production_wells):
    total_net_cost = 0
    total_sequestration_capacity = sum(well['max_sequestration_rate'] for well in production_wells)
    
    for year in range(year_start, year_end + 1, 5):
        print(f"Year: {year}")
        row = data[(data['Type'] == 'Onshore') & (data['Year'] == year)]
        print(f"Filtered Data for Year {year}:")
        print(row)
        if not row.empty:
            row = row.iloc[0]
            revenue = row.get('Sale_price', 1)
            capture_cost = row.get('Cost_capture', 1)
            transport_cost = row.get('Cost_transport', 1) * pipeline_length
            storage_cost = row.get('Cost_storage', 1)
            print("rev", revenue, capture_cost, transport_cost, storage_cost)
            total_cost = capture_cost + transport_cost + storage_cost
            print(total_cost)
            net_cost = total_cost - revenue
            total_net_cost += net_cost * total_sequestration_capacity
            print(net_cost, total_net_cost)
    
    print("ccs", total_net_cost)
    return total_net_cost

def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime_start, plant_lifetime_end, co2_production
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate net costs
    total_net_ccs_cost = calculate_net_ccs_cost(ccs_data, plant_lifetime_start, plant_lifetime_end, pipeline_length, production_wells)

    # Decision
    if decommissioning_cost < total_net_ccs_cost:
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f}, "
                f"as it is lower than the net costs of CCS (${total_net_ccs_cost:,.2f}).")

# Compare options and provide insights
result = compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, PLANT_LIFETIME_START, PLANT_LIFETIME_END, CO2_PRODUCTION
)

print(result)


decom 43771500
Year: 2025
Filtered Data for Year 2025:
      Type  Year  Cost for capture /tCO2 ($)  Cost of Transport /tCO2/km ($)  \
0  Onshore  2025                       113.0                             2.2   

   Cost of Storage /tCO2 ($)  Effective Sale Price /tCO2 ($)  
0                       34.1                            39.0  
rev 1 1 3 1
5
4 16000000.0
Year: 2030
Filtered Data for Year 2030:
      Type  Year  Cost for capture /tCO2 ($)  Cost of Transport /tCO2/km ($)  \
1  Onshore  2030                       105.0                             2.0   

   Cost of Storage /tCO2 ($)  Effective Sale Price /tCO2 ($)  
1                       29.5                            55.0  
rev 1 1 3 1
5
4 32000000.0
Year: 2035
Filtered Data for Year 2035:
      Type  Year  Cost for capture /tCO2 ($)  Cost of Transport /tCO2/km ($)  \
2  Onshore  2035                        92.0                             1.8   

   Cost of Storage /tCO2 ($)  Effective Sale Price /tCO2 ($)  
2            

In [59]:
import pandas as pd

# Load the data
carbon_data_path = 'carbon.csv'
hydrogen_data_path = 'hydrogen.csv'
carbon_df = pd.read_csv(carbon_data_path)
hydrogen_df = pd.read_csv(hydrogen_data_path)

# Define constants
CO2_PRODUCTION = 1.2e6  # 1.2 million tonnes per annum
PLANT_LIFETIME_START = 2034
PLANT_LIFETIME_END = 2055

# Decommissioning costs
export_pipeline_cost = 275000 * 20 # Cost for 400 km pipeline
export_riser_cost = 11500 * 15  # Cost for 80 m riser
production_wells = [{
    'well_cost': 6900000,
    'well_cost_unit': 1,
    'pipeline_cost': 210000,
    'pipeline_length': 2,
    'riser_cost': 10500,
    'riser_length': 15,
    'requires_pipeline': True,
    'max_sequestration_rate': 2e6  # Maximum sequestration rate per well in tonnes per annum
},
{
    'well_cost': 4400000,
    'well_cost_unit': 1,
    'pipeline_cost': 230000,
    'pipeline_length': 1,
    'riser_cost': 11500,
    'riser_length': 15,
    'requires_pipeline': True,
    'max_sequestration_rate': 2e6  # Maximum sequestration rate per well in tonnes per annum
}]
jacket_cost = 15750000 # Cost for 2 jackets
jacket_units = 1  # Number of jackets
topside_modules = [
    {'module_cost': 1575000, 'unit_count': 1},
    {'module_cost': 2750000, 'unit_count': 1},
    {'module_cost': 2100000, 'unit_count': 1},
    {'module_cost': 3069000, 'unit_count': 1},
    {'module_cost': 575000, 'unit_count': 1},
   
]
misc_costs = 0  # Misc0ellaneous costs

# Define functions
def calculate_decommissioning_cost(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
):
    total_well_costs = 0
    pipeline_length = 0
    
    for well in production_wells:
        well_cost = well['well_cost'] * well['well_cost_unit']
        pipeline_length += well['pipeline_length'] if well['requires_pipeline'] else 0
        pipeline_cost = well['pipeline_cost'] * well['pipeline_length'] if well['requires_pipeline'] else 0
        riser_cost = well['riser_cost'] * well['riser_length'] if well['requires_pipeline'] else 0
        total_well_costs += well_cost + pipeline_cost + riser_cost

    total_jacket_costs = jacket_cost * jacket_units
    total_topside_costs = sum(module['module_cost'] * module['unit_count'] for module in topside_modules)

    total_cost = export_pipeline_cost + export_riser_cost + total_well_costs + total_jacket_costs + total_topside_costs + misc_costs
    print("decom",total_cost)
    return total_cost, pipeline_length

def calculate_net_ccs_cost(data, year_start, year_end, pipeline_length, production_wells):
    total_net_cost = 0
    total_sequestration_capacity = sum(well['max_sequestration_rate'] for well in production_wells)
    
    for year in range(year_start, year_end + 1):
        # print(year)
        yearx=year-year%5

        row = data[(data['Type'] == 'Offshore') & (data['Year'] == yearx)]
        if not row.empty:
            row = row.iloc[0]
            revenue = row.get('Sale_price', 1)
            capture_cost = row.get('Cost_capture', 1)
            transport_cost = row.get('Cost_transport', 1) * pipeline_length
            storage_cost = row.get('Cost_storage', 1)
            # print("rev",revenue,capture_cost,transport_cost,storage_cost)
            total_cost = capture_cost + transport_cost + storage_cost
            # print(total_cost)
            net_cost = total_cost - revenue
            total_net_cost += net_cost * total_sequestration_capacity
            # print(net_cost,total_net_cost)
    print("ccs",total_net_cost)
    return total_net_cost

def calculate_net_hydrogen_cost(data, year_start, year_end, pipeline_length, installed_capacity=1000, operating_hours_per_year=2080):
    total_net_cost = 0
    for year in range(year_start, year_end + 1, 5):
        yearx=year-year%5
        for tech in ['Alkaline', 'PEM']:
            row = data[(data['Type'] == tech) & (data['Year'] == yearx)]
            if not row.empty:
                row = row.iloc[0]
                sell_price = row.get('Sell Price / kg', 0)
                production_cost = row.get('Cost of Production /kg ($) (Operational)', 0)
                transport_cost_per_100km = row.get('Cost of Transport /kg/100km', 0)
                capital_cost_per_kw = row.get('Cost for Installation /kW ($) (Capital Cost)', 0)
                efficiency = row.get('Efficiency (kWh/kgH2)', 1)
                
                # Calculate transport cost based on pipeline length
                transport_cost = transport_cost_per_100km * (pipeline_length / 100)
                
                # Calculate annual hydrogen production
                annual_hydrogen_production = installed_capacity * operating_hours_per_year / efficiency
                
                # Capital cost should be considered for the initial year only
                capital_cost = capital_cost_per_kw * installed_capacity if year == year_start else 0

                # Calculate total cost per kg of hydrogen
                total_cost = production_cost + transport_cost + capital_cost / annual_hydrogen_production

                # Calculate net cost
                net_cost_per_kg = total_cost - sell_price
                total_net_cost += net_cost_per_kg * annual_hydrogen_production
    print("h2",total_net_cost)
    return total_net_cost


def compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    ccs_data, hydrogen_data, plant_lifetime_start, plant_lifetime_end, co2_production
):
    # Calculate decommissioning cost and pipeline length
    decommissioning_cost, pipeline_length = calculate_decommissioning_cost(
        export_pipeline_cost, export_riser_cost,
        production_wells, jacket_cost, jacket_units, topside_modules, misc_costs
    )
    
    # Calculate net costs
    total_net_ccs_cost = calculate_net_ccs_cost(ccs_data, plant_lifetime_start, plant_lifetime_end, pipeline_length, production_wells)
    total_net_hydrogen_cost = calculate_net_hydrogen_cost(hydrogen_data, plant_lifetime_start, plant_lifetime_end, pipeline_length)

    # Decision
    if decommissioning_cost < total_net_ccs_cost and decommissioning_cost  < total_net_hydrogen_cost:
        return (f"Decommissioning is preferred with a cost of ${decommissioning_cost:,.2f}, "
                f"as it is lower than the net costs of CCS (${total_net_ccs_cost:,.2f}) and Hydrogen (${total_net_hydrogen_cost:,.2f}).")

    if total_net_ccs_cost < total_net_hydrogen_cost:
        return (f"CCS is preferred with a net cost of ${total_net_ccs_cost:,.2f}. "
                f"Consider onshore CCS for lower capture costs over time. The net cost analysis shows that CCS is more cost-effective "
                f"compared to Hydrogen production.")
    else:
        return (f"Hydrogen is preferred with a net cost of ${total_net_hydrogen_cost:,.2f}. "
                f"Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers "
                f"a more favorable balance of production and transport costs, yielding lower net costs compared to PEM technology.")

# Compare options and provide insights
result = compare_options(
    export_pipeline_cost, export_riser_cost,
    production_wells, jacket_cost, jacket_units, topside_modules, misc_costs,
    carbon_df, hydrogen_df, PLANT_LIFETIME_START, PLANT_LIFETIME_END, CO2_PRODUCTION
)

print(result)

decom 43771500
ccs 352000000.0
h2 -204648.62602179238
Hydrogen is preferred with a net cost of $-204,648.63. Consider Alkaline Hydrogen for better profitability and efficiency. Hydrogen production via Alkaline technology offers a more favorable balance of production and transport costs, yielding lower net costs compared to PEM technology.
